In [1]:
import numpy as np
from numpy.random import Generator, PCG64
from deap import base
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

_CONFIG = {
    'init_size': 100,
    'max_size': 1000,
    'n_generations': 50,
    'n_traits': 3,
    'trait_min': -1,
    'trait_max': 1,
    'p_mut': 0.1,
    'mut_std': 0.1,
    'init_opt_gen': np.zeros(3),
    'sel_std': 0.95,
    'warm_rate': 0.01,
    'competition_rate': 0.1,
    'seed': 2137
}

rng = Generator(PCG64(_CONFIG['seed']))

def fitness(population, optimal_traits=_CONFIG['init_opt_gen'], sel_std=_CONFIG['sel_std']):
    distances = np.linalg.norm(population - optimal_traits, axis=1)
    return np.exp(-distances / (2 * sel_std**2))

def mutate(population, p_mut=_CONFIG['p_mut'], mut_std=_CONFIG['mut_std']):

    for i, individual in enumerate(population):
        if rng.uniform(0, 1) < p_mut:
            ind = rng.integers(0, individual.size)
            mut_val = rng.normal(0, mut_std**2)
            individual[ind] += mut_val

    return population

def reproduce(parents):
    n = len(parents)
    n_half = n // 2
    rng.shuffle(parents)
    group1, group2 = parents[:n_half], parents[n_half:]
    offspring = []
    for p1, p2 in zip(group1, group2):
        child = (p1 + p2) / rng.uniform(2, 4)
        offspring.append(child)
    return offspring

def select_parents(population, fitnesses, comp_rate=_CONFIG['competition_rate']):
    parents = [element for element, prob in zip(population, fitnesses) if rng.uniform(0, 1) < (1 - _CONFIG['competition_rate']) * prob]
    return parents

def init_individual(size=_CONFIG['n_traits'], trait_min=_CONFIG['trait_min'], trait_max=_CONFIG['trait_max']):
    return rng.uniform(trait_min, trait_max, size)

def init_population(pcls, ind_init, size=_CONFIG['init_size']):
    return pcls(ind_init() for _ in range(size))

def global_warming(optimal_traits, warm_rate=_CONFIG['warm_rate']):
    return optimal_traits + rng.uniform(0, warm_rate, _CONFIG['n_traits'])

def meteor_hit(trait_min=_CONFIG['trait_min'], trait_max=_CONFIG['trait_max']):
    return rng.uniform(trait_min, trait_max, _CONFIG['n_traits'])

def simulate_evolution(config):
    toolbox = create_toolbox()
    population = init_population(list, init_individual, config['init_size'])
    optimal_traits = config['init_opt_gen']

    stat_categories = ["population", "fitnesses", "optimal_traits"]
    pop_history = pd.DataFrame(columns=stat_categories, index=np.arange(config['n_generations']))
    history = dict(stat_categories=[])

    for gen in range(config['n_generations']):

        history["population"] = population
        history["optimal_traits"] = optimal_traits

        population = toolbox.mutate(population)

        fitnesses = toolbox.evaluate(population, optimal_traits)

        history["fitnesses"] = fitnesses

        parents = toolbox.select(population, fitnesses)

        offspring = toolbox.mate(parents)

        population = parents + offspring

        optimal_traits = global_warming(optimal_traits)

        if rng.uniform(0, 1) < 0.005:
            optimal_traits = meteor_hit()
            print(f"Meteor hit at generation {gen}.")

        pop_history.loc[gen] = history.copy()

        if len(population) > config['max_size']:
            fitnesses = toolbox.evaluate(population, optimal_traits)
            indcs = np.argsort(fitnesses)
            population = [population[i] for i in indcs[:config['max_size']]]

        elif len(population) <= 0:
            print(f"Population died out at generation {gen}.")
            break


    pop_history.dropna(how='all', axis=0, inplace=True)
    return pop_history

def create_toolbox():
    toolbox = base.Toolbox()
    toolbox.register("evaluate", fitness)
    toolbox.register("mate", reproduce)
    toolbox.register("mutate", mutate)
    toolbox.register("select", select_parents)
    return toolbox


SyntaxError: invalid syntax (base.py, line 187)